In [1]:
import os


# Changer le répertoire actuel
os.chdir('/kaggle/input/e2cnn/e2cnn-master')

# Vérifier le répertoire actuel
print("Répertoire actuel :", os.getcwd())


Répertoire actuel : /kaggle/input/e2cnn/e2cnn-master


# General E(2)-Equivariant Steerable CNNs  -  A concrete example


In [2]:
import torch

from e2cnn import gspaces
from e2cnn import nn

Finally, we build a **Steerable CNN** and try it MNIST.

Let's also use a group a bit larger: we now build a model equivariant to $8$ rotations.
We indicate the group of $N$ discrete rotations as $C_N$, i.e. the **cyclic group** of order $N$.
In this case, we will use $C_8$.

Because the inputs are still gray-scale images, the input type of the model is again a *scalar field*.

However, internally we use *regular fields*: this is equivalent to a *group-equivariant convolutional neural network*.

Finally, we build *invariant* features for the final classification task by pooling over the group using *Group Pooling*.

The final classification is performed by a two fully connected layers.

# The model

Here is the definition of our model:

In [3]:
class C8SteerableCNN(torch.nn.Module):

    def __init__(self, n_classes=10):

        super(C8SteerableCNN, self).__init__()

        self.mask = torch.ones(1, 1, 28, 28)  # Adjust the shape to match your input dimensions
        self.mask = self.mask.to(device)

        # the model is equivariant under rotations by 45 degrees, modelled by C8
        self.r2_act = gspaces.Rot2dOnR2(N=8)

        # the input image is a scalar field, corresponding to the trivial representation
        in_type = nn.FieldType(self.r2_act, [self.r2_act.trivial_repr])

        # we store the input type for wrapping the images into a geometric tensor during the forward pass
        self.input_type = in_type

        # convolution 1
        # first specify the output type of the convolutional layer
        # we choose 24 feature fields, each transforming under the regular representation of C8
        out_type = nn.FieldType(self.r2_act, 24*[self.r2_act.regular_repr])
        self.block1 = nn.SequentialModule(
            nn.MaskModule(in_type, 29, margin=1),
            nn.R2Conv(in_type, out_type, kernel_size=7, padding=1, bias=False),
            nn.InnerBatchNorm(out_type),
            nn.ReLU(out_type, inplace=True)
        )

        # convolution 2
        # the old output type is the input type to the next layer
        in_type = self.block1.out_type
        # the output type of the second convolution layer are 48 regular feature fields of C8
        out_type = nn.FieldType(self.r2_act, 48*[self.r2_act.regular_repr])
        self.block2 = nn.SequentialModule(
            nn.R2Conv(in_type, out_type, kernel_size=5, padding=2, bias=False),
            nn.InnerBatchNorm(out_type),
            nn.ReLU(out_type, inplace=True)
        )
        self.pool1 = nn.SequentialModule(
            nn.PointwiseAvgPoolAntialiased(out_type, sigma=0.66, stride=2)
        )

        # convolution 3
        # the old output type is the input type to the next layer
        in_type = self.block2.out_type
        # the output type of the third convolution layer are 48 regular feature fields of C8
        out_type = nn.FieldType(self.r2_act, 48*[self.r2_act.regular_repr])
        self.block3 = nn.SequentialModule(
            nn.R2Conv(in_type, out_type, kernel_size=5, padding=2, bias=False),
            nn.InnerBatchNorm(out_type),
            nn.ReLU(out_type, inplace=True)
        )

        # convolution 4
        # the old output type is the input type to the next layer
        in_type = self.block3.out_type
        # the output type of the fourth convolution layer are 96 regular feature fields of C8
        out_type = nn.FieldType(self.r2_act, 96*[self.r2_act.regular_repr])
        self.block4 = nn.SequentialModule(
            nn.R2Conv(in_type, out_type, kernel_size=5, padding=2, bias=False),
            nn.InnerBatchNorm(out_type),
            nn.ReLU(out_type, inplace=True)
        )
        self.pool2 = nn.SequentialModule(
            nn.PointwiseAvgPoolAntialiased(out_type, sigma=0.66, stride=2)
        )

        # convolution 5
        # the old output type is the input type to the next layer
        in_type = self.block4.out_type
        # the output type of the fifth convolution layer are 96 regular feature fields of C8
        out_type = nn.FieldType(self.r2_act, 96*[self.r2_act.regular_repr])
        self.block5 = nn.SequentialModule(
            nn.R2Conv(in_type, out_type, kernel_size=5, padding=2, bias=False),
            nn.InnerBatchNorm(out_type),
            nn.ReLU(out_type, inplace=True)
        )

        # convolution 6
        # the old output type is the input type to the next layer
        in_type = self.block5.out_type
        # the output type of the sixth convolution layer are 64 regular feature fields of C8
        out_type = nn.FieldType(self.r2_act, 64*[self.r2_act.regular_repr])
        self.block6 = nn.SequentialModule(
            nn.R2Conv(in_type, out_type, kernel_size=5, padding=1, bias=False),
            nn.InnerBatchNorm(out_type),
            nn.ReLU(out_type, inplace=True)
        )
        self.pool3 = nn.PointwiseAvgPoolAntialiased(out_type, sigma=0.66, stride=1, padding=0)

        self.gpool = nn.GroupPooling(out_type)

        # number of output channels
        c = self.gpool.out_type.size

        # Fully Connected
        self.fully_net = torch.nn.Sequential(
            torch.nn.Linear(c, 64),
            torch.nn.BatchNorm1d(64),
            torch.nn.ELU(inplace=True),
            torch.nn.Linear(64, n_classes),
        )

    def forward(self, input: torch.Tensor):
        # wrap the input tensor in a GeometricTensor
        # (associate it with the input type)
        x = nn.GeometricTensor(input, self.input_type)

        # apply each equivariant block

        # Each layer has an input and an output type
        # A layer takes a GeometricTensor in input.
        # This tensor needs to be associated with the same representation of the layer's input type
        #
        # The Layer outputs a new GeometricTensor, associated with the layer's output type.
        # As a result, consecutive layers need to have matching input/output types
        x = self.block1(x)
        x = self.block2(x)
        x = self.pool1(x)

        x = self.block3(x)
        x = self.block4(x)
        x = self.pool2(x)

        x = self.block5(x)
        x = self.block6(x)

        # pool over the spatial dimensions
        x = self.pool3(x)

        # pool over the group
        x = self.gpool(x)

        # unwrap the output GeometricTensor
        # (take the Pytorch tensor and discard the associated representation)
        x = x.tensor

        # classify with the final fully connected layers)
        x = self.fully_net(x.reshape(x.shape[0], -1))

        return x

Let's try the model on *rotated* MNIST

In [4]:
import torch
from torch.utils.data import DataLoader, TensorDataset

def merge_loaders(train_loaders, test_loaders, batch_size):
    # Combine train data
    train_images, train_labels = [], []
    for loader in train_loaders:
        for images, labels in loader['loader']:
            train_images.append(images)
            train_labels.append(labels)

    # Combine all train tensors into one
    train_images = torch.cat(train_images, dim=0)
    train_labels = torch.cat(train_labels, dim=0)

    # Create a single training dataset
    train_dataset = TensorDataset(train_images, train_labels)
    combined_train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    # Combine test data
    test_images, test_labels = [], []
    for loader in test_loaders:
        for images, labels in loader['loader']:
            test_images.append(images)
            test_labels.append(labels)

    # Combine all test tensors into one
    test_images = torch.cat(test_images, dim=0)
    test_labels = torch.cat(test_labels, dim=0)

    # Create a single test dataset
    test_dataset = TensorDataset(test_images, test_labels)
    combined_test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    return combined_train_loader, combined_test_loader

In [5]:
import torch.utils.data
import torchvision
import torchvision.transforms.functional as F
from torchvision.transforms import Compose, Pad, ToTensor, Normalize, ToPILImage
from PIL import Image

import numpy as np


class RotatedMNISTDataset(torch.utils.data.Dataset):
    '''
        This class provides MNIST images with random rotations sampled from
        a list of rotation angles. This list is dependent of the number of tasks
        `num_tasks` and the distance (measured in degrees) between tasks
        `per_task_rotation`.
    '''
    def __init__(self, root, train=True, transform=None, download=True, num_tasks=5, per_task_rotation=45):
        self.dataset = torchvision.datasets.MNIST(root=root, train=train, transform=transform, download=download)
        self.transform = transform
        self.rotation_angles = []
        for task in range(num_tasks):
            self.rotation_angles.append(float((task) * per_task_rotation))

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        image, label = self.dataset[idx]
        angle = np.random.choice(self.rotation_angles)  # Randomly choose a rotation angle
        rotated_image = F.rotate(image, angle, fill=(0,))


        return rotated_image, label, angle

In [6]:
def tasks_rotMNIST(num_tasks,
                   per_task_rotation,
                   batch_size,
                   transform=[]
                   ):
    '''
    returns:
    - train_loaders: List of dictionaries containing
        * the data loader
        * the task number
        * the corresponding rotation angle
    - test_loaders: List of dictionaries containing
        * the data loader
        * the task number
        * the corresponding rotation angle
    '''

    class RotationTransform:
        def __init__(self, angle):
            self.angle = angle

        def __call__(self, x):
            return F.rotate(x, self.angle, fill=(0,))

    train_loaders = []
    test_loaders = []

    g = torch.Generator()
    g.manual_seed(0)  # check: always setting generator to 0 ensures the same ordering of data


    for task in range(num_tasks):
        rotation_degree = (task) * per_task_rotation

        extended_transform = transform.copy()
        extended_transform.extend([
            Pad((0, 0, 1, 1)),  # Add padding
            ToTensor(),
            Normalize((0.1307,), (0.3081,))  # Normalize
        ])
        transforms = Compose(extended_transform)
        #print(transforms)

        train = torchvision.datasets.MNIST('~/mnist_data/', train=True, download=True, transform=transforms)
        test = torchvision.datasets.MNIST('~/mnist_data/', train=False, download=True, transform=transforms)

        train_loader = torch.utils.data.DataLoader(train,  batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True, generator=g)
        test_loader = torch.utils.data.DataLoader(test,  batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True, generator=g)

        train_loaders.append({
            'loader':train_loader,
            'task':task,
            'rot':rotation_degree})
        test_loaders.append({
            'loader':test_loader,
            'task':task,
            'rot':rotation_degree})

    return train_loaders, test_loaders

In [7]:
# Step 2: Set parameters for the new dataset
num_tasks = 8  # Number of rotation tasks
per_task_rotation = 45  # Degrees of rotation per task
batch_size = 64  # Batch size for training/testing

# Step 3: Load the rotated MNIST dataset
train_loaders, test_loaders = tasks_rotMNIST(
    num_tasks=num_tasks,
    per_task_rotation=per_task_rotation,
    batch_size=batch_size
)

combined_train_loader, combined_test_loader = merge_loaders(
    train_loaders, test_loaders, batch_size=batch_size
)

for images, labels in combined_train_loader:
    print(f"Input image shape after transform: {images.shape}")  # Should be [batch_size, 1, 29, 29]
    break


Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:00<00:00, 14471778.25it/s]


Extracting /root/mnist_data/MNIST/raw/train-images-idx3-ubyte.gz to /root/mnist_data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 469632.87it/s]


Extracting /root/mnist_data/MNIST/raw/train-labels-idx1-ubyte.gz to /root/mnist_data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:00<00:00, 4423481.02it/s]


Extracting /root/mnist_data/MNIST/raw/t10k-images-idx3-ubyte.gz to /root/mnist_data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 3207159.73it/s]


Extracting /root/mnist_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to /root/mnist_data/MNIST/raw

Input image shape after transform: torch.Size([64, 1, 29, 29])


In [8]:
from torch.utils.data import Dataset
from torchvision.transforms import RandomRotation
from torchvision.transforms import Pad
from torchvision.transforms import Resize
from torchvision.transforms import ToTensor
from torchvision.transforms import Compose

import numpy as np

from PIL import Image

device = 'cuda' if torch.cuda.is_available() else 'cpu'


Build the dataset

In [9]:
class MnistRotDataset(Dataset):

    def __init__(self, mode, transform=None):
        assert mode in ['train', 'test']

        if mode == "train":
            file = "mnist_rotation_new/mnist_all_rotation_normalized_float_train_valid.amat"
        else:
            file = "mnist_rotation_new/mnist_all_rotation_normalized_float_test.amat"

        self.transform = transform

        data = np.loadtxt(file, delimiter=' ')

        self.images = data[:, :-1].reshape(-1, 28, 28).astype(np.float32)
        self.labels = data[:, -1].astype(np.int64)
        self.num_samples = len(self.labels)

    def __getitem__(self, index):
        image, label = self.images[index], self.labels[index]
        image = Image.fromarray(image)
        if self.transform is not None:
            image = self.transform(image)
        return image, label

    def __len__(self):
        return len(self.labels)

# images are padded to have shape 29x29.
# this allows to use odd-size filters with stride 2 when downsampling a feature map in the model
pad = Pad((0, 0, 1, 1), fill=0)

# to reduce interpolation artifacts (e.g. when testing the model on rotated images),
# we upsample an image by a factor of 3, rotate it and finally downsample it again
resize1 = Resize(87)
resize2 = Resize(29)

totensor = ToTensor()

Let's build the model

In [10]:
model = C8SteerableCNN().to(device)

/kaggle/input/e2cnn/e2cnn-master/e2cnn/nn/modules/r2_conv/basisexpansion_singleblock.py:80: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at /usr/local/src/pytorch/aten/src/ATen/native/IndexingUtils.h:27.)
  full_mask[mask] = norms.to(torch.uint8)


The model is now randomly initialized.
Therefore, we do not expect it to produce the right class probabilities.

However, the model should still produce the same output for rotated versions of the same image.
This is true for rotations by multiples of $\frac{\pi}{2}$, but is only approximate for rotations by $\frac{\pi}{4}$.

Let's test it on a random test image:
we feed eight rotated versions of the first image in the test set and print the output logits of the model for each of them.

In [11]:

def test_model(model: torch.nn.Module, x: Image):
    # evaluate the `model` on 8 rotated versions of the input image `x`
    model.eval()

    wrmup = model(torch.randn(1, 1, 29, 29).to(device))
    del wrmup

    x = resize1(pad(x))

    print()
    print('##########################################################################################')
    header = 'angle |  ' + '  '.join(["{:6d}".format(d) for d in range(10)])
    print(header)
    with torch.no_grad():
        for r in range(8):
            x_transformed = totensor(resize2(x.rotate(r*45., Image.BILINEAR))).reshape(1, 1, 29, 29)
            x_transformed = x_transformed.to(device)

            y = model(x_transformed)
            y = y.to('cpu').numpy().squeeze()

            angle = r * 45
            print("{:5d} : {}".format(angle, y))
    print('##########################################################################################')
    print()



The output of the model is already almost invariant.
However, we still observe small fluctuations in the outputs.

This is because the model contains some operations which might break equivariance.
For instance, every convolution includes a padding of $2$ pixels per side. This is adds information about the actual orientation of the grid where the image/feature map is sampled because the padding is not rotated with the image.

During training, the model will observe rotated patterns and will learn to ignore the noise coming from the padding.

So, let's train the model now.
The model is exactly the same used to train a normal *PyTorch* architecture:

In [12]:
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=5e-5, weight_decay=1e-5)

In [14]:
import csv

# Création ou ouverture d'un fichier CSV
csv_file = "/kaggle/working/accuracy_log.csv"
with open(csv_file, mode="w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["Epoch", "Test Accuracy"])  # En-tête du fichier

# Entraînement du modèle
for epoch in range(11):
    model.train()
    for i, (x, t) in enumerate(combined_train_loader):

        optimizer.zero_grad()

        x = x.to(device)
        t = t.to(device)

        y = model(x)

        loss = loss_function(y, t)

        loss.backward()

        optimizer.step()

    total = 0
    correct = 0
    with torch.no_grad():
        model.eval()
        for i, (x, t) in enumerate(combined_test_loader):
            x = x.to(device)
            t = t.to(device)

            y = model(x)

            _, prediction = torch.max(y.data, 1)
            total += t.shape[0]
            correct += (prediction == t).sum().item()

    # Calcul de l'accuracy
    test_accuracy = correct / total * 100.0
    print(f"Epoch {epoch} | Test Accuracy: {test_accuracy:.2f}%")

    # Ajout des résultats au fichier CSV
    with open(csv_file, mode="a", newline="") as file:
        writer = csv.writer(file)
        writer.writerow([epoch, test_accuracy])

Epoch 0 | Test Accuracy: 97.99%
Epoch 1 | Test Accuracy: 97.82%
Epoch 2 | Test Accuracy: 98.02%
Epoch 3 | Test Accuracy: 98.12%
Epoch 4 | Test Accuracy: 98.18%
Epoch 5 | Test Accuracy: 98.01%
Epoch 6 | Test Accuracy: 97.87%
Epoch 7 | Test Accuracy: 98.26%
Epoch 8 | Test Accuracy: 98.04%
Epoch 9 | Test Accuracy: 98.04%
Epoch 10 | Test Accuracy: 98.05%


In [ ]:
# retrieve the first image from the test set
x, y = next(iter(raw_mnist_test))


# evaluate the model
test_model(model, x)